# Ocean Color

## Import packages

In [ ]:
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymannkendall as mk
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
start = "1997-09"
stop = "2023-03"

# Variable to analyse
variables = ["chlor_a", "Rrs_443", "Rrs_560"]
assert set(variables) <= {
    "chlor_a",
    "Rrs_412",
    "Rrs_443",
    "Rrs_490",
    "Rrs_510",
    "Rrs_560",
    "Rrs_665",
}

# Regions to plot
regions = {
    "Global": {"lon_slice": slice(-180, 180), "lat_slice": slice(90, -90)},
    "50S-50N": {"lon_slice": slice(-180, 180), "lat_slice": slice(50, -50)},
    "NASTG": {"lon_slice": slice(-80, 0), "lat_slice": slice(50, 0)},
}
for region, slices in regions.items():
    # Enforce sorting as original data
    for k, v in slices.items():
        assert v.start >= v.stop if k == "lat_slice" else v.start <= v.stop, (region, k)

## Define request

In [ ]:
collection_id = "satellite-ocean-colour"

request = {
    "projection": "regular_latitude_longitude_grid",
    "version": "6_0",
    "format": "zip",
}

## Functions to cache

In [ ]:
def regionalised_spatial_weighted_mean(ds, variable, lon_slice, lat_slice):
    da = ds[variable]
    da = utils.regionalise(da, lon_slice=lon_slice, lat_slice=lat_slice)
    if variable == "chlor_a":
        da = da.where((da > 1.0e-3) & (da < 1.0e2))
    with xr.set_options(keep_attrs=True):
        da = 10 ** diagnostics.spatial_weighted_mean(np.log10(da))
    da.attrs["long_name"] = da.attrs["long_name"].replace(" (not log-transformed)", "")
    return da.to_dataset(name=variable)

## Download and transform data

In [ ]:
datasets = []
for variable in variables:
    for region, slices in regions.items():
        requests = download.update_request_date(
            request
            | {
                "variable": "remote_sensing_reflectance"
                if variable.startswith("Rrs")
                else "mass_concentration_of_chlorophyll_a"
            },
            start=start,
            stop=stop,
            stringify_dates=True,
        )
        ds = download.download_and_transform(
            collection_id,
            requests,
            transform_func=regionalised_spatial_weighted_mean,
            transform_func_kwargs={"variable": variable} | slices,
            chunks={"year": 1, "month": 1, "variable": 1},
        )
        datasets.append(ds.expand_dims(latitudes=[region]))
ds = xr.merge(datasets).compute()

## Define plotting functions

In [ ]:
def plot_timeseries(da):
    fig, ax = plt.subplots(1, 1)

    # Compute variables
    da_daily = da.resample(time="D").mean()
    da_yearly = da.resample(time="Y").mean()
    da_running = (
        da.resample(time="MS")
        .mean()
        .rolling(time=48, center=True, min_periods=1)
        .mean()
    )
    *_, slope, intercept = mk.original_test(da_yearly.squeeze())

    # Plots lines
    da_daily.plot(label="daily", color="tab:grey", ls=" ", marker=".", ax=ax)
    da_running.plot(
        add_legend=False,
        label="48-month running",
        color="tab:red",
        ls="--",
        marker=" ",
        ax=ax,
    )
    ax.plot(
        da_yearly["time"],
        np.arange(da_yearly.sizes["time"]) * slope + intercept,
        label="yearly slope",
        color="tab:blue",
        ls="-",
        marker=" ",
    )

    # Plot shades
    missions = {
        "SeaWiFS": ["1998-01-01", "2010-12-31"],
        "MERIS": ["2002-03-01", "2012-04-30"],
    }
    for i, (mission, xlims) in enumerate(missions.items()):
        plt.axvspan(
            *pd.to_datetime(xlims).tolist(),
            facecolor=matplotlib.colors.colorConverter.to_rgba(f"C{i}", alpha=0.2),
            edgecolor=matplotlib.colors.colorConverter.to_rgba(f"C{i}", alpha=1),
            lw=1.5,
            ls="--",
            label=mission,
        )

    # Final settings
    ax.set_ylim(da.quantile([0.01, 0.99]).values.tolist())
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.grid()
    return fig, ax

## Plot timeseries

In [ ]:
for variable in variables:
    da = ds[variable]
    for latitdues, da_lats in da.groupby("latitudes"):
        fig, ax = plot_timeseries(da_lats)
        plt.show()